<a href="https://colab.research.google.com/github/tazar09/scrap_imobiliar/blob/main/scrap_imobilar%5B06-feb2024%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
# import nltk
# from nltk.tokenize import word_tokenize
# nltk.download('punkt')
import re
import datetime
from google.colab import files

In [ ]:
# sapun solid
# pasta de dinti
# bioderma
# apa micerala
# betisoare de urechi
# SAPUN Lichid theo

Request web-sites

In [ ]:
zf_url = 'https://www.zf.ro/'
zf_response = requests.get(zf_url)
zf_soup = BeautifulSoup(zf_response.content, 'html.parser')

profit_url = 'https://www.profit.ro/'
profit_response = requests.get(profit_url)
profit_soup = BeautifulSoup(profit_response.content, 'html.parser')

economica_url = 'https://www.economica.net/ultima-ora'
economica_response = requests.get(economica_url)
economica_soup = BeautifulSoup(economica_response.content, 'html.parser')

1. zf.ro

In [ ]:
 lst = []
 link = []
 for x in zf_soup.find_all('h3'):
   x_text = x.a.text
   x_link = x.a.get('href')
   lst.append(x_text)
   link.append(x_link)

In [ ]:
df = pd.DataFrame({'title': lst, 'link': link})
df.shape

(136, 2)

In [ ]:
df['link'] = 'zf.ro' + df['link']

In [ ]:
df['link'][1]

'zf.ro/business-international/una-dintre-cele-mai-vechi-firme-poloneze-si-germania-ultimele-atrase-22254937'

In [ ]:
lst = ['chiri', 'imobiliar', 'rezidențial',
       'Colliers', 'One', 'Dragoteanu', 'Euroest', 'Property', 'Forty', 'RealKom', 'Storia', 'Storia.ro', 'Angheloiu',
       'estate', 'real-estate', 'REAL-ESTATE' 'residence',
       'dezvoltator','ciment', 'beton', 'construcț', 'autorizaț', 'locuinț', 'apartament', 'case', 'euro/mp', 'cartier',
       ]

regex_imobiliare = r'\b\w*(?:' + '|'.join(lst) + r')\w*\b'

In [ ]:
def selector(x):
  matches = re.findall(regex_imobiliare, x)
  if matches:
    return True
  else:
    return False

In [ ]:
df['new_check']=df['title'].apply(selector)

In [ ]:
# numar_gasit = df['new_check'].value_counts()[1]
# numar_gasit

In [ ]:
df = df[df['new_check'] == True]

In [ ]:
excludere_words = ['Airbnb', 'Skanska', 'restaur', 'electronice', 'preziden', 'vaccin', 'Charles', 'comerț', 'software', 'serviciilor', 'eroport', 'cafea', 'arom', 'reconstru', 'Petrom', 'OnePlus', 'CNAIR']
regex_excludere= r'\b\w*(?:' + '|'.join(excludere_words) + r')\w*\b'

def excludere(x):
  matches = re.findall(regex_excludere, x)
  if matches:
    return True
  else:
    return False

In [ ]:
df['excludere'] = df['title'].apply(excludere)

<ipython-input-14-aacbe12d722f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['excludere'] = df['title'].apply(excludere)


In [ ]:
df = df[df['excludere'] == False]

In [ ]:
df['drop'] = df['link'].str.extract(r'-(\d+)$')

In [ ]:
df = df.drop_duplicates(subset = 'drop')

Add timedate to the scrapping

In [ ]:
now = datetime.datetime.now()
timestamp = now.strftime("%d-%m-%Y")
timestamp

'06-02-2024'

In [ ]:
df['timestamp'] = timestamp
df['idnumber'] = df['drop'] + df['timestamp'].str.replace('-', '')

Make final df by appending

In [ ]:
df_zf = df
df_zf = pd.concat([df_zf, df], axis = 0)
df_zf = df_zf.drop_duplicates('idnumber')


In [ ]:
df_zf = df_zf.drop(['new_check', 'excludere'], axis = 1)

----------------------------------------------------------------------------

2. profit.ro

In [ ]:
divs = profit_soup.find_all('div', class_ = 'row')
div_2 = profit_soup.find_all('div', class_ = 'col-md-4')

In [ ]:
lst2 = []
link2 = []

for div in div_2:
  a_tags = div.find_all('a')
  for a in a_tags:
    title = a.get('href')
    text = a.text
    lst2.append(title)
    link2.append(text)

In [ ]:
df2 = pd.DataFrame({'link': lst2, 'title': link2})
df2['check'] = df2['title'].str.strip() == ''

In [ ]:
df2 = df2[df2['check'] == False].drop('check', axis = 1)

In [ ]:
df2['link'] = 'profit.ro' + df2['link']

In [ ]:
df2 = df2[['title', 'link']]

In [ ]:
df2['drop'] = df2['link'].str.extract(r'-(\d+)$')

In [ ]:
now2 = datetime.datetime.now()
timestamp2 = now2.strftime('%d-%m-%Y')
df2['timestamp'] = timestamp2

In [ ]:
df2['idnumber'] = df2['drop'] + df2['timestamp'].str.replace('-', '')

In [ ]:
df2['new_check'] = df2['title'].apply(selector)

In [ ]:
df_profit = df2[df2['new_check'] == True]

In [ ]:
df_profit['excludere'] = df_profit['title'].apply(excludere)
df_profit = df_profit[df_profit['excludere'] == False]

<ipython-input-33-6cb8c6796e34>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_profit['excludere'] = df_profit['title'].apply(excludere)


In [ ]:
df_profit = df_profit.drop(['new_check', 'excludere'], axis = 1)

In [ ]:
df_profit.head(199)

,title,link,drop,timestamp,idnumber
38,Cum se construiesc noi cartiere rezidențiale,profit.ro/povestile-bursei/cum-se-construiesc-...,21402196,06-02-2024,2140219606022024
95,Vânzările rezidențiale ale One United Properti...,profit.ro/povesti-cu-profit/real-estate-constr...,21483987,06-02-2024,2148398706022024
153,2024 începe într-o notă optimistă pentru dezvo...,profit.ro/legal/2024-incepe-intr-o-nota-optimi...,21468978,06-02-2024,2146897806022024


In [ ]:
df2.columns

Index(['title', 'link', 'drop', 'timestamp', 'idnumber', 'new_check'], dtype='object')

In [ ]:
columnnames = ['title', 'link', 'drop', 'timestamp', 'idnumber']
df_daily = pd.DataFrame(columns = columnnames)
df_daily

,title,link,drop,timestamp,idnumber


In [ ]:
df_daily = pd.concat([df_zf, df_profit], axis = 0)
df_daily

,title,link,drop,timestamp,idnumber
21,Distribuitorul de bunuri de larg consum Licuri...,zf.ro/companii/distribuitorul-bunuri-larg-cons...,22253086,06-02-2024,2225308606022024
35,"ZF Live. Gabriel Blăniţă, Colliers România: Ce...",zf.ro/zf-live/zf-live-gabriel-blanita-colliers...,22234566,06-02-2024,2223456606022024
45,ANALIZĂ ZF Cum să locuieşti mai bine. Scumpire...,zf.ro/cum-sa-locuiesti-mai-bine/analiza-zf-loc...,22253050,06-02-2024,2225305006022024
59,Dezvoltatorul imobiliar One United Properties ...,zf.ro/zf-24/dezvoltatorul-imobiliar-one-united...,22253991,06-02-2024,2225399106022024
61,Ministerul Economiei aduce clarificări cu priv...,zf.ro/burse-fonduri-mutuale/ministerul-economi...,22253037,06-02-2024,2225303706022024
70,Grafic: Evoluţia preţurilor pentru câteva dint...,zf.rohttp://www.zfcorporate.ro/rd-22254617,22254617,06-02-2024,2225461706022024
102,Ministerul Economiei explică insolvenţa Romaer...,zf.ro/burse-fonduri-mutuale/ministerul-economi...,22253171,06-02-2024,2225317106022024
117,"Dragos Dragoteanu, Euroest Invest: Dan Şucu, n...",zf.ro/opinii/dragos-dragoteanu-euroest-invest-...,22239993,06-02-2024,2223999306022024
132,Cât de prost merg lucrurile în România? Anul t...,zf.ro/constructii-imobiliare/cat-prost-merg-lu...,22240413,06-02-2024,2224041306022024
38,Cum se construiesc noi cartiere rezidențiale,profit.ro/povestile-bursei/cum-se-construiesc-...,21402196,06-02-2024,2140219606022024


In [ ]:
current_date = datetime.datetime.now().strftime("%d_%m_%Y")

In [ ]:
path_google = f"/content/imob_scrap_{current_date}.csv"
df_daily.to_csv(path_google, index=False, encoding = 'utf-8-sig')
files.download(path_google)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

______________________________________________________

3. economica.net

In [ ]:
lst3 = []
link3 = []
for x in economica_soup.find_all('div', class_ = 'article add-border article--xs-list' ):


  h2_element = x.find('h2', class_ = 'article__title font-weight-700')
  h2_text = h2_element.get_text()
  h2_href = h2_element.find('a')['href']


  lst3.append(h2_text)
  link3.append(h2_href)

In [ ]:
df3 = pd.DataFrame({'title': lst3, 'link': link3})
df3.head()

,title,link
0,\n„Autostrada Sudului”: Traseul drumului de ma...,https://www.economica.net/autostrada-sudului-t...
1,"\nDr. Max, Dona, Regina Maria, MedLife îşi con...",https://www.economica.net/dr-max-dona-regina-m...
2,\nLicitația pentru centurile Azuga și Bușteni ...,https://www.economica.net/licitatia-pentru-cen...
3,\nO curtea de apel federală a decis să resping...,https://www.economica.net/o-curtea-de-apel-fed...
4,\nPrim-ministrul georgian Kobahidze acuză Ucra...,https://www.economica.net/prim-ministrul-georg...


In [ ]:
economica_url = 'https://www.economica.net/imobiliare'
economica_response = requests.get(economica_url)
economica_soup = BeautifulSoup(economica_response.content, 'html.parser')

In [ ]:
lst4 = []
link4 = []
for x in economica_soup.find_all('div', class_ = 'article add-border article--xs-list' ):


  h2_element = x.find('h2', class_ = 'article__title font-weight-700')
  h2_text = h2_element.get_text()
  h2_href = h2_element.find('a')['href']


  lst4.append(h2_text)
  link4.append(h2_href)

In [ ]:
lst5 = []
link5 = []

for x in economica_soup.find_all('div', class_ = 'article__date'):
  date = x.get_text()
  lst5.append(date)


In [ ]:
df_econ1 = pd.DataFrame(lst4, columns = ['titlu'])
df_econ2 = pd.DataFrame(link4, columns = ['link'])
df_econ3 = pd.DataFrame(lst5, columns = ['date'])


In [ ]:
df_economica = pd.concat([df_econ1, df_econ2, df_econ3], axis = 1)
df_economica

,titlu,link,date
0,\nNicuşor Dan acuză o lacună legislativă în ca...,https://www.economica.net/nicusor-dan-acuza-o-...,04 feb. 2024
1,"\nHanul Solacolu, care a fost prima fabrică de...",https://www.economica.net/hanul-solacolu-conso...,02 feb. 2024
2,\nNoua Casă 2024: Guvernul a aprobat un plafon...,https://www.economica.net/guvernul-a-aprobat-u...,31 ian. 2024
3,\n83% dintre proprietarii de locuințe din Româ...,https://www.economica.net/83-dintre-proprietar...,31 ian. 2024
4,\nComisia Europeană vrea o nouă derogare pentr...,https://www.economica.net/comisia-europeana-vr...,30 ian. 2024
5,\nIonuț Stan a fost promovat Partener al Cross...,https://www.economica.net/ionut-stan-a-fost-pr...,30 ian. 2024
6,\nCum să îmbunătățeşti performanța agenției ta...,https://www.economica.net/cum-sa-imbunatatesti...,30 ian. 2024
7,\nMitiska vrea un nou portofoliu de parcuri de...,https://www.economica.net/mitiska-romania-pite...,28 ian. 2024
8,"\nCum va arăta Țiriac City, mega-proiectul spo...",https://www.economica.net/tiriac-city-complex-...,28 ian. 2024
9,\nArcese România a închiriat 10.500 de mp în C...,https://www.economica.net/arcese-romania-a-inc...,25 ian. 2024


In [ ]:
len(lst5)

30

In [ ]:
len(lst4)

30

In [ ]:
len(link4)

30

In [ ]:
romanian_sentiment_lexicon = {
    'bun': 1,
    'frumos': 1,
    'fericit': 1,
    'bine': 1,
    'rău': -1,
    'urât': -1,
    'trist': -1,
    'greu': -1,
    'crescut': 1,
    'creșter': 1
}

In [ ]:
pip install nltk.textblob

ERROR: Could not find a version that satisfies the requirement nltk.textblob (from versions: none)
ERROR: No matching distribution found for nltk.textblob


In [ ]:
article_text = 'Volumul lucrărilor de construcții a crescut, în primele cinci luni, față de perioada similară din 2022, ca serie brută, cu 11,3%, iar ca serie ajustată în funcție de numărul de zile lucrătoare și de sezonalitate, cu 12,3%, arată datele publicate de Institutul Național de Statistică (INS). ”Volumul lucrărilor de construcții, ca serie brută, a crescut, pe total, cu 11,3%. Pe elemente de structură, creșteri au avut loc la lucrările de reparații capitale (+32,5%), lucrările de construcții noi (+11,3%) și la lucrările de întreținere și reparații curente (+4%). Pe obiecte de construcții, au avut loc creșteri la construcțiile inginerești (+31,8%)”, arată datele INS. Volumul lucrărilor de construcții a crescut, ca serie ajustată în funcție de numărul de zile lucrătoare și de sezonalitate, cu 12,3%, creștere evidențiată la lucrările de reparații capitale (+31%), lucrările de construcții noi (+11,7%) și la lucrările de întreținere și reparații curente (+2,6%). Pe obiecte de construcții, creșteri au avut loc la construcțiile inginerești (+30,8%). Clădirile rezidențiale și clădirile nerezidențiale au scăzut cu 1,9%, respectiv cu 1,5%. Mai 2023 comparativ cu aprilie 2023 Volumul lucrărilor de construcții a crescut, ca serie brută, cu 19,9%, creștere evidențiată la lucrările de reparații capitale (+33,6%), lucrările de întreținere și reparații curente (+28,1%) și la lucrările de construcții noi (+15,3%). Pe obiecte de construcții, au avut loc creșteri la construcții inginerești (+23,2%), clădirile nerezidențiale (+21,7%) și la clădirile rezidențiale (+10,8%).Volumul lucrărilor de construcții a crescut, ca serie ajustată în funcție de numărul de zile lucrătoare și de sezonalitate, cu 3,7%, creștere evidențiată la lucrările de întreținere și reparații curente (+10,7%) și la lucrările de reparații capitale (+8,1%). Lucrările de construcții noi au scăzut cu 0,1%. Pe obiecte de construcții, volumul lucrărilor de construcții a crescut la clădirile nerezidențiale (+9,1%), construcțiile inginerești (+2,6%) și la clădirile rezidențiale (+2,4%). Mai 2023 comparativ cu mai 2022 Volumul lucrărilor de construcții ca serie brută a crescut pe total cu 10,7%. Pe elemente de structură, creșteri s-au înregistrat la lucrările de reparații capitale (+41,7%), lucrările de întreținere și reparații curente (+9,1%) și la lucrările de construcții noi (+7,4%). Pe obiecte de construcții, creștere s-a înregistrat la construcțiile ingineresti (+40,5%). Clădirile rezidențiale și clădirile nerezidențiale au scăzut cu 11,9%, respectiv cu 6,9%.Ca serie ajustată în funcție de numărul de zile lucrătoare și de sezonalitate, volumul lucrărilor de construcții a crescut, pe total, cu 11,4%. Pe elemente de structură s-au evidențiat creșteri la lucrările de reparații capitale (+43,2%), lucrările de întreținere și reparații curente (+8,7%) și la lucrările de construcții noi (+7,4%). Pe obiecte de construcții, creștere s-a înregistrat la construcțiile inginerești (+41,7%). Clădirile rezidențiale și clădirile nerezidențiale au scăzut cu 12,0%, respectiv cu 7,3%.'

pip install nltk
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

stemmer = SnowballStemmer('romanian')

def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove punctuation
    table = str.maketrans('', '', string.punctuation)
    tokens = [word.translate(table) for word in tokens]

    # Remove stopwords and perform stemming
    stop_words = set(stopwords.words('romanian'))
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]

    return tokens

tokens = preprocess_text(article_text)

def calculate_sentiment_score(tokens, sentiment_lexicon):
    sentiment_score = 0
    for word in tokens:
        if word in sentiment_lexicon:
            sentiment_score += sentiment_lexicon[word]

    return sentiment_score

sentiment_score = calculate_sentiment_score(tokens, romanian_sentiment_lexicon)

def determine_sentiment(sentiment_score):
    if sentiment_score > 0:
        return "Positive"
    elif sentiment_score < 0:
        return "Negative"
    else:
        return "Neutral"

sentiment = determine_sentiment(sentiment_score)

print("Sentiment Score:", sentiment_score)
print("Sentiment:", sentiment)

SyntaxError: invalid syntax (<ipython-input-53-462e3f60f0a6>, line 3)

In [ ]:
nltk.download('wordnet')